In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# ⚙️ Configuración del navegador
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(
    service=Service("C:/Users/alber/OneDrive/Desktop/Proyectos/Baratazo/ChromeDriver/chromedriver-win64/chromedriver.exe"),
    options=options
)

# 🌐 Cargar la página
driver.get("https://compraonline.bonpreuesclat.cat/search?q=queso%20sin%20lactosa")

# 🍪 Aceptar cookies
try:
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
    ).click()
    print("✅ Cookies aceptadas")
except:
    print("⚠️ No aparecieron cookies")

# ⏳ Esperar a que aparezca el primer producto
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CLASS_NAME, "product-card-container"))
)

# 📜 Scroll general rápido para cargar todos los productos
prev = -1
reintentos = 0
MAX_REINTENTOS = 5

while reintentos < MAX_REINTENTOS:
    cards = driver.find_elements(By.CLASS_NAME, "product-card-container")
    print(f"📦 Productos detectados: {len(cards)}")
    if len(cards) == prev:
        reintentos += 1
    else:
        reintentos = 0
    prev = len(cards)
    driver.execute_script("window.scrollBy(0, 1000);")
    time.sleep(0.4)

print("✅ Scroll general terminado")

# 📄 Lectura con scroll individual y recuperación dinámica
productos = []
vistos = set()

for idx in range(prev):
    try:
        # 🧠 Reobtener el elemento justo antes de usarlo para evitar stale references
        element = driver.find_elements(By.CLASS_NAME, "product-card-container")[idx]
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
        time.sleep(0.3)

        nombre = element.find_element(By.CSS_SELECTOR, '[data-test="fop-title"]').text.strip()
        presentacion = element.find_element(By.CSS_SELECTOR, '[data-test="fop-size"]').text.strip()
        identificador = f"{nombre} - {presentacion}"
        if identificador in vistos:
            continue

        try:
            precio = element.find_element(By.CSS_SELECTOR, '[data-test="fop-price"]').text.strip()
        except:
            precio = "N/A"
        try:
            precio_ud = element.find_element(By.CSS_SELECTOR, '[data-test="fop-price-per-unit"]').text.strip()
        except:
            precio_ud = "N/A"

        productos.append({
            "Nombre": nombre,
            "Precio": precio,
            "Precio/unidad": precio_ud,
            "Presentación": presentacion
        })
        vistos.add(identificador)

    except Exception as e:
        print(f"⚠️ Error en producto {idx} → {e}")

# 🧾 Crear DataFrame
df = pd.DataFrame(productos)


✅ Cookies aceptadas
📦 Productos detectados: 30
📦 Productos detectados: 30
📦 Productos detectados: 54
📦 Productos detectados: 60
📦 Productos detectados: 60
📦 Productos detectados: 60
📦 Productos detectados: 60
📦 Productos detectados: 60
📦 Productos detectados: 60
✅ Scroll general terminado


In [ ]:
pd.set_option('display.max_rows', None)
df

,Nombre,Precio,Precio/unidad,Presentación
0,TGT Formatge mozzarella sense lactosa,"1,99 €","(13,27 € per quilo)","150g\n(13,27 € per quilo)"
1,BONPREU Formatge gouda sense lactosa,"2,70 €","(13,50 € per quilo)","200g\n(13,50 € per quilo)"
2,ARLA Formatge untar,"2,15 €","(10,75 € per quilo)","200g\n(10,75 € per quilo)"
3,GARCIA BAQUERO Formatge sense lactosa,"3,05 €","(20,33 € per quilo)","150g\n(20,33 € per quilo)"
4,KAIKU Formatge Emmental ratllat sense lactosa,"1,99 €","(19,90 € per quilo)","100g\n(19,90 € per quilo)"
5,EL PASTOR Formatge de cabra sense lactosa,"3,90 €","(21,67 € per quilo)","180g\n(21,67 € per quilo)"
6,PHILADELPHIA Formatge fos natural light,"2,55 €","(12,75 € per quilo)","200g\n(12,75 € per quilo)"
7,EL VENTERO Formatge tendre light,"3,39 €","(21,19 € per quilo)","160g\n(21,19 € per quilo)"
8,EL VENTERO Formatge tendre reduït en sal,"3,39 €","(21,19 € per quilo)","160g\n(21,19 € per quilo)"
9,BURGO DE ARIAS Formatge fresc sense lactosa,"3,29 €","(15,23 € per quilo)","3 x 72g\n(15,23 € per quilo)"
